In [24]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset, DatasetDict
import pandas as pd
from sklearn.model_selection import train_test_split


In [31]:
# -------------------------------
# 1. Load datasets
# -------------------------------
# CSV format must have: "text" and "label"
# label = int (0=Left, 1=Center, 2=Right)  OR (0–4 for 5 classes)
# for now it will only have these 2 columns but later on it will have article company and their normal bias rating also attached

df= pd.read_csv("non_indian_articles.csv") #non indian
df2 = pd.read_csv("indian_articles.csv")
print(df.columns)
df.head()

Index(['Source', 'MBFC URL', 'Bias', 'Country', 'Factual Reporting',
       'Media Type', 'Source URL', 'Credibility', 'Source ID#', 'Unnamed: 9'],
      dtype='object')


,Source,MBFC URL,Bias,Country,Factual Reporting,Media Type,Source URL,Credibility,Source ID#,Unnamed: 9
0,11 News - KKCO,https://mediabiasfactcheck.com/11-news-kkco/,Least Biased,USA,High,TV Station,nbc11news.com,High,100865,NaN
1,13 News - WIBW,https://mediabiasfactcheck.com/13-news-wibw/,Least Biased,USA,High,TV Station,wibw.com,High,100877,NaN
2,1440 Newsletter,https://mediabiasfactcheck.com/1440-newsletter/,Least Biased,USA,High,Website,join1440.com,High,100834,NaN
3,1945 -19fortyfive,https://mediabiasfactcheck.com/1945-19fortyfiv...,Right-Center,USA,Mostly Factual,Website,19fortyfive.com,High,107240,NaN
4,2020 Conservative,https://mediabiasfactcheck.com/2020-conservati...,Questionable,Unknown,Very Low,Website,2020conservative.com,Low,105611,NaN


In [33]:
df2.drop(columns=["Unnamed: 9","Source ID#", "Factual Reporting","Source"], axis=1, inplace=True)

df2.head()

,MBFC URL,Bias,Country,Media Type,Source URL,Credibility
0,https://mediabiasfactcheck.com/biotech-express...,Conspiracy-Pseudoscience,India,Magazine,biotechexpressmag.com,Low
1,https://mediabiasfactcheck.com/boomlive-bias/,Right-Center,India,Website,boomlive.in,High
2,https://mediabiasfactcheck.com/business-standa...,Right-Center,India,Newspaper,business-standard.com,High
3,https://mediabiasfactcheck.com/the-hindu-busin...,Right-Center,India,Newspaper,thehindubusinessline.com,Medium
4,https://mediabiasfactcheck.com/india-tv/,Right-Center,India,TV Station,indiatvnews.com,High


In [35]:
df2["MBFC URL"].to_list()[1]

'https://mediabiasfactcheck.com/boomlive-bias/'

In [9]:
df['Bias'].value_counts()

Bias
Questionable                2269
Least Biased                2076
Left-Center                 1830
Right-Center                 829
Pro-Science                  697
Left                         649
Conspiracy-Pseudoscience     572
Right                        509
Satire                       168
Conspiracy-Pseuscience         2
Name: count, dtype: int64

In [ ]:
def map_bias(label):
    if label in ["Left", "Left-Center"]:
        return "Left"
    elif label in ["Right", "Right-Center"]:
        return "Right"
    elif label == "Least Biased":
        return "Center"
    else:
        return None

In [18]:
df["MappedBias"] = df["Bias"].apply(map_bias)

# Drop unwanted rows
df = df.dropna(subset=["MappedBias"])

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


C:\Users\2077a\AppData\Local\Temp\ipykernel_19212\2919727027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["MappedBias"] = df["Bias"].apply(map_bias)


In [20]:
df["MappedBias"].value_counts()

MappedBias
Left      2479
Center    2076
Right     1338
Name: count, dtype: int64

In [21]:
df.head()

,Source,MBFC URL,Bias,Country,Factual Reporting,Media Type,Source URL,Credibility,Source ID#,MappedBias
0,11 News - KKCO,https://mediabiasfactcheck.com/11-news-kkco/,Least Biased,USA,High,TV Station,nbc11news.com,High,100865,Center
1,13 News - WIBW,https://mediabiasfactcheck.com/13-news-wibw/,Least Biased,USA,High,TV Station,wibw.com,High,100877,Center
2,1440 Newsletter,https://mediabiasfactcheck.com/1440-newsletter/,Least Biased,USA,High,Website,join1440.com,High,100834,Center
3,1945 -19fortyfive,https://mediabiasfactcheck.com/1945-19fortyfiv...,Right-Center,USA,Mostly Factual,Website,19fortyfive.com,High,107240,Right
6,23WIFR News,https://mediabiasfactcheck.com/23wifr-news/,Least Biased,USA,High,TV Station,wifr.com,High,100890,Center


In [ ]:
# Combine datasets (Step A + Step B later)
# full_df = pd.concat([non_indian_df, indian_df], ignore_index=True)

# -------------------------------
# 2. Train/Validation/Test split
# -------------------------------
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    full_df["text"], full_df["label"], test_size=0.3, stratify=full_df["label"], random_state=42
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, stratify=temp_labels, random_state=42
)

train_df = pd.DataFrame({"text": train_texts, "label": train_labels})
val_df = pd.DataFrame({"text": val_texts, "label": val_labels})
test_df = pd.DataFrame({"text": test_texts, "label": test_labels})

# -------------------------------
# 3. Tokenizer
# -------------------------------
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

train_ds = Dataset.from_pandas(train_df).map(tokenize, batched=True)
val_ds = Dataset.from_pandas(val_df).map(tokenize, batched=True)
test_ds = Dataset.from_pandas(test_df).map(tokenize, batched=True)

# Hugging Face DatasetDict
dataset = DatasetDict({
    "train": train_ds,
    "validation": val_ds,
    "test": test_ds
})

# Format for PyTorch
for split in dataset:
    dataset[split].set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# -------------------------------
# 4. Load model
# -------------------------------
num_labels = len(full_df["label"].unique())  # auto-detect number of classes
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)

# -------------------------------
# 5. Training setup
# -------------------------------
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",   # evaluate after each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,   # keep best checkpoint
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,            # can stop earlier with early stopping
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=50,
    metric_for_best_model="accuracy",  # choose metric to track
    save_total_limit=2,                 # only keep 2 checkpoints
)

# -------------------------------
# 6. Define Trainer
# -------------------------------
from evaluate import load
accuracy = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"]}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # stop if no improvement for 2 evals
)

# -------------------------------
# 7. Train model
# -------------------------------
trainer.train()

# -------------------------------
# 8. Evaluate on test set
# -------------------------------
metrics = trainer.evaluate(dataset["test"])
print("Final test metrics:", metrics)

# -------------------------------
# 9. Save final model
# -------------------------------
trainer.save_model("./roberta_bias_final")
tokenizer.save_pretrained("./roberta_bias_final")
